# Propensity score matching

This script does the following:

1. Import the necessary libraries
2. Loads the data into a Pandas DataFrame
3. Define the treatment and control groups
4. Define the features used to predict the treatment
5. Create an instance of LRSRegressor to estimate propensity scores
6. Fit the model to the data
7. Compute the propensity scores
8. Create an instance of PropensityScoreMatching
9. Perform the matching
10. Inspect the matched data

It is important to note that propensity score matching is a powerful tool, but it should be used with caution. There is a lot of assumptions that needs to be met before applying the technique, and it is crucial to validate the assumptions and check the balance of the treated and control groups after the matching.

Here's a script that demonstrates how you might perform propensity score matching in Python using the library "causalml":


## Import libraries

In [ ]:
# Import necessary libraries
import os
os.environ['USE_PYGEOS'] = '0'
import sys
import pandas as pd
# from causalml.inference.meta import LRSRegressor
# from causalml.propensity import PropensityScoreMatching
# from causalml.propensity import ElasticNetPropensityModel
# from causalml.match import NearestNeighborMatch, create_table_one
import psmpy as ps
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import geopandas as gpd
import seaborn as sns
from pathlib import Path
from tableone import TableOne
from utils import read_data, plot_covariate_distributions, plot_match, compare_balance, sizeof_fmt, optimize_memory_df, plot_categorical_proportional_diff, compute_mean_differences_and_proportions, love_plot, sensitivity_analysis_k_neighbors

In [ ]:
data_folder = Path('../Data/')
res_folder = Path('../Results/')
model_folder = res_folder/'Models'

## Import data

In [ ]:
# Load data into a Pandas DataFrame
data = read_data("../Data/processed/full_dataset_nonull.parquet.gzip")

In [ ]:
data.groupby('NOANNEE').uuid.nunique()

In [ ]:
data.gp.value_counts()

In [ ]:
data = gpd.GeoDataFrame(data, crs = 4326, geometry=gpd.points_from_xy(data.lon_masked, data.lat_masked))
data = data.to_crs(2056)
data['E'], data['N'] = data['geometry'].x, data['geometry'].y
data['E:N'] = data['E']*data['N']

In [ ]:
data_final = data.copy()

In [ ]:
del data

In [ ]:
# data_final = data[data.treatment.isnull()==False]
data_final['DEDUCTIBLE_above_500'] = 1
data_final.loc[data_final.MTFRANCHISECOUV < 500, 'DEDUCTIBLE_above_500'] = 0

data_final['gp'] = data_final['gp'].astype(str)
# data_final['CANTON'] = data_final.filter(regex='CANTON_ACRONU').idxmax(axis=1).str.replace('CANTON_NAME_', '')
data_final['SEX'] = data_final.filter(regex='SEX_').idxmax(axis=1).str.replace('SEX_', '')

In [ ]:
data_final[data_final.gp.isin(['LCA & AOS','AOS only','LCA only'])].gp.value_counts(normalize = True)

In [ ]:
data_final[(data_final.A10A_ATC_N > 0)|(data_final.A10B_ATC_N > 0)].gp.value_counts(normalize = True)

## Evolution of groups over time

In [ ]:
cnt_by_gp = pd.DataFrame(data_final.groupby(['NOANNEE','gp']).size(), columns = ['n']).reset_index()

In [ ]:
cnt_by_gp = cnt_by_gp.pivot(index = 'NOANNEE', values = 'n', columns = 'gp').reset_index()

In [ ]:
cnt_by_gp

In [ ]:
y = [cnt_by_gp['AOS only'],
              cnt_by_gp['LCA & AOS'],
              cnt_by_gp['LCA only'],
              cnt_by_gp['No insurance'],
              cnt_by_gp['No usage'],
              cnt_by_gp['No usage - No AOS & not insured LCA'],
              cnt_by_gp['No usage - No LCA & not insured AOS'],
              cnt_by_gp['Not insured AOS, LCA'],
              cnt_by_gp['Not insured LCA, AOS']]

In [ ]:
#create area chart
plt.stackplot(cnt_by_gp.NOANNEE, y, labels =  cnt_by_gp.drop('NOANNEE', axis = 1).columns)
plt.legend(loc='lower left', fontsize = 8)

In [ ]:
# data_2017.to_csv('../Data/processed/data_2017.csv', index = False)

### Question to consider : I am not including the individuals that didn't claim anything within a year. Should they be included in the control group?

In [ ]:
df_treated = optimize_memory_df(data_final[data_final.treatment.isnull()==False])

In [ ]:
df_treated.groupby('NOANNEE').uuid.nunique()

In [ ]:
del data_final

### Filter out individuals that do not belong to the treatment and control for the whole 5 years

In [ ]:
# Get unique years
unique_years = set(df_treated['NOANNEE'])

# Group by patient_id and filter
df_treated_filtered = df_treated.groupby('uuid').filter(lambda x: set(x['NOANNEE']) == unique_years)

In [ ]:
df_treated_filtered.groupby('NOANNEE').uuid.nunique()

### Create yearly datasets

In [ ]:
data_2017 = df_treated[df_treated.NOANNEE == 2017]
data_2018 = df_treated[df_treated.NOANNEE == 2018]
data_2019 = df_treated[df_treated.NOANNEE == 2019]
data_2020 = df_treated[df_treated.NOANNEE == 2020]
data_2021 = df_treated[df_treated.NOANNEE == 2021]

In [ ]:
### All uuid present for 5 years
data_2017_filtered = df_treated_filtered[df_treated_filtered.NOANNEE == 2017]
data_2018_filtered = df_treated_filtered[df_treated_filtered.NOANNEE == 2018]
data_2019_filtered = df_treated_filtered[df_treated_filtered.NOANNEE == 2019]
data_2020_filtered = df_treated_filtered[df_treated_filtered.NOANNEE == 2020]
data_2021_filtered = df_treated_filtered[df_treated_filtered.NOANNEE == 2021]

In [ ]:
corr_atc_amount_treatment = data_2021.filter(regex = 'Diabetes|Epilespy|Rheumatologic conditions|Hyperlipidemia|Thyroid disorders|treatment').corr()['treatment'].sort_values()

## Reduce memory usage

In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

## Create subsample for tests

## Using scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Define the treatment variable and covariates
id_var = 'uuid'
treatment_var = 'treatment'
outcome_var = 'PRESTATIONS_BRUTES_AOS'
outcome_vars = ['DRUGAMOUNT_BRUT','PRESTATIONS_BRUTES_AOS','PRESTATIONS_NETTES_AOS','PRESTATIONS_BRUTES_LCA','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH','PRESTATIONS_ACCIDENT','PRESTATIONS_TOTAL','n_atc','NBRE_FACTURES_LCA', 'NBRE_FACTURES_AOS', 'NBRE_FACTURES_TOTAL', 'n_inpatient_hosp',
 'n_outpatient_hosp',
 'n_month_outpatienthosp',
 'n_month_inpatienthosp',
 'time_to_rehosp_in',
 'time_to_rehosp_out']
categorical_columns = ['SEX_F','LANG_FR','LANG_DE','LANG_IT','LANG_EN','CANTON_ACRONYM_AG',
 'CANTON_ACRONYM_AI',
 'CANTON_ACRONYM_AR',
 'CANTON_ACRONYM_BE',
 'CANTON_ACRONYM_BL',
 'CANTON_ACRONYM_BS',
 'CANTON_ACRONYM_FR',
 'CANTON_ACRONYM_GE',
 'CANTON_ACRONYM_GL',
 'CANTON_ACRONYM_GR',
 'CANTON_ACRONYM_JU',
 'CANTON_ACRONYM_LU',
 'CANTON_ACRONYM_NE',
 'CANTON_ACRONYM_NW',
 'CANTON_ACRONYM_OW',
 'CANTON_ACRONYM_SG',
 'CANTON_ACRONYM_SH',
 'CANTON_ACRONYM_SO',
 'CANTON_ACRONYM_SZ',
 'CANTON_ACRONYM_TG',
 'CANTON_ACRONYM_TI',
 'CANTON_ACRONYM_UR',
 'CANTON_ACRONYM_VD',
 'CANTON_ACRONYM_VS',
 'CANTON_ACRONYM_ZG',
 'CANTON_ACRONYM_ZH']
continuous_columns = ['NBAGE','cds','ssep2','mean_ndvi','MTFRANCHISECOUV', 'mean_lst', 'mean_pm10', 'mean_pm25', 'mean_no2', 'mean_carnight', 'E', 'N', 'E:N', 'D_MEDIC_B', 'D_MEDIC_S']

In [ ]:
def compute_propensity_scores(df, treatment_var, outcome_vars, categorical_columns, continuous_columns):
    """
    This function computes propensity scores and returns a copy of the original dataframe with an additional 
    column for the propensity scores.

    Parameters:
        df (pd.DataFrame): The dataframe for which propensity scores are to be computed.
        treatment_var (str): The name of the treatment variable in df.
        outcome_vars (list of str): The names of the outcome variables in df.
        categorical_columns (list of str): The names of the categorical variables in df.
        continuous_columns (list of str): The names of the continuous variables in df.

    Returns:
        df_scaled (pd.DataFrame): A copy of the original dataframe with an additional 'propensity_score' column.
    """
    # Initialize a StandardScaler
    scaler = StandardScaler()
    
    # Copy the dataframe
    df_scaled = df[continuous_columns+categorical_columns+outcome_vars+[treatment_var]+[id_var]].copy()
    
    # Scale the continuous variables
    if len(continuous_columns) > 0:
        df_scaled[continuous_columns] = scaler.fit_transform(df_scaled[continuous_columns])
    
    # Fill NaN values in 'D_MEDIC_S' and 'D_MEDIC_B' with their respective mean
    if 'D_MEDIC_S' in df_scaled.columns:
        df_scaled['D_MEDIC_S'] = df_scaled['D_MEDIC_S'].fillna(df_scaled['D_MEDIC_S'].mean())
    if 'D_MEDIC_B' in df_scaled.columns:
        df_scaled['D_MEDIC_B'] = df_scaled['D_MEDIC_B'].fillna(df_scaled['D_MEDIC_B'].mean())
    
    # Fit a logistic regression model to estimate propensity scores
    logistic_model = LogisticRegression(max_iter=1000)
    logistic_model.fit(df_scaled[categorical_columns+continuous_columns], df_scaled[treatment_var])

    formula = treatment_var + ' ~ ' + ' + '.join(categorical_columns + continuous_columns)
    logit_model = smf.logit(formula, data=df_scaled).fit()
    
    # Store model results
    params = logit_model.params
    conf = logit_model.conf_int()
    conf['Odds Ratio'] = params
    conf.columns = ['2.5%', '97.5%', 'Odds Ratio']
    # convert log odds to ORs
    odds = pd.DataFrame(np.exp(conf))
    # check if pvalues are significant
    odds['pvalues'] = logit_model.pvalues
    odds['significant?'] = ['significant' if pval <= 0.05 else 'not significant' for pval in logit_model.pvalues]
    updated_odds = update_variable_names(odds, variable_names, 'odds')
   
    # Compute propensity scores
    propensity_scores = logistic_model.predict_proba(df_scaled[categorical_columns+continuous_columns])[:, 1]
    
    # Calculate the c-statistic (area under the ROC curve)
    c_statistic = roc_auc_score(df_scaled[treatment_var], propensity_scores)
    print("C-statistic:", c_statistic)
    
    # Add propensity scores to the dataset
    df_scaled['propensity_score'] = propensity_scores
    
    return df_scaled, updated_odds


In [ ]:
variable_names = pd.DataFrame({"old": ['NBAGE',"NBAGE_std", "ssep2_std",'ssep2', "SEX_F",'SEX','LANG', "cds_std",'cds','LANG_FR','D_MEDIC_B','D_MEDIC_S','D_MEDIC_B_std','D_MEDIC_S_std','DEDUCTIBLE_above_500','E_std','N_std','E_std:N_std','PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','AMBULATOIRE','STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH','MTFRANCHISECOUV','mean_pm10','mean_no2','mean_pm25','mean_ndvi','mean_lst','mean_carnight'],
                           "new": ['Age',"Age", "SES index",'SES index', "Sex (Female)",'Sex','Langage', "CDS",'CDS','French speaker','Access to prim. care med.','Access to spec. med.','Access to prim. care med.','Access to spec. med.','Franchise (>500)','E','N','E:N','Montant tot. prestations','Montant tot. prestations (AOS)','Montant tot. prestations (LCA)','Montant tot. ambulatoire','Montant tot. stationnaire','Montant tot. accident','Montant tot. maladie','Montant tot. maternité','Franchise','PM10','NO2','PM25','NDVI','LST','Nighttime car noise']})
def update_variable_names(summary_table, variable_names, table_type):
    name_mapper = variable_names.set_index('old')['new'].to_dict()
    if table_type == 'summary':
        name_mapper = {f"{key}, mean (SD)": f"{value}, mean (SD)" for key, value in name_mapper.items()}

    summary_table = summary_table.rename(index=name_mapper)
    return summary_table

In [ ]:
df_psm_2017, odds_2017 = compute_propensity_scores(data_2017, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2018, odds_2018 = compute_propensity_scores(data_2018, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2019, odds_2019 = compute_propensity_scores(data_2019, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2020, odds_2020 = compute_propensity_scores(data_2020, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2021, odds_2021 = compute_propensity_scores(data_2021, treatment_var, outcome_vars, categorical_columns, continuous_columns)

In [ ]:
df_psm_2017_filtered, odds_2017_filtered = compute_propensity_scores(data_2017_filtered, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2018_filtered, odds_2018_filtered = compute_propensity_scores(data_2018_filtered, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2019_filtered, odds_2019_filtered = compute_propensity_scores(data_2019_filtered, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2020_filtered, odds_2020_filtered = compute_propensity_scores(data_2020_filtered, treatment_var, outcome_vars, categorical_columns, continuous_columns)
df_psm_2021_filtered, odds_2021_filtered = compute_propensity_scores(data_2021_filtered, treatment_var, outcome_vars, categorical_columns, continuous_columns)

In [ ]:
def forest_plot(df_odds, folder, period):

    if not os.path.exists(folder):
        os.makedirs(folder)
    fig, ax = plt.subplots(nrows=1, sharex=True, sharey=True, figsize=(6, 10))
    for idx, row in df_odds.iloc[::-1].iterrows():
        ci = [[row['Odds Ratio'] - row[::-1]['2.5%']], [row['97.5%'] - row['Odds Ratio']]]
        if row['significant?'] == 'significant':
            if row['Odds Ratio'] > 1:
                plt.errorbar(x=[row['Odds Ratio']], y=[row.name], xerr=ci,
                    ecolor='tab:red', capsize=3, linestyle='None', linewidth=1, marker="o", 
                             markersize=5, mfc="tab:red", mec="tab:red")
            else:
                plt.errorbar(x=[row['Odds Ratio']], y=[row.name], xerr=ci,
                    ecolor='tab:blue', capsize=3, linestyle='None', linewidth=1, marker="o", 
                             markersize=5, mfc="tab:blue", mec="tab:blue")
        else:
            plt.errorbar(x=[row['Odds Ratio']], y=[row.name], xerr=ci,
                ecolor='tab:gray', capsize=3, linestyle='None', linewidth=1, marker="o", 
                         markersize=5, mfc="tab:gray", mec="tab:gray")
    plt.axvline(x=1, linewidth=0.8, linestyle='--', color='black')
    plt.tick_params(axis='both', which='major', labelsize=8)
    plt.xlabel('Odds Ratio and 95% Confidence Interval', fontsize=8)
    plt.tight_layout()
    plt.savefig(folder/'forest_plot_{}.png'.format(period))
    plt.show()
model_directory = model_folder /'Determinants of integrative medicine'/ 'Logistic regression'

In [ ]:
forest_plot(odds_2017, model_directory, '2017')
forest_plot(odds_2018, model_directory, '2018')
forest_plot(odds_2021, model_directory, '2021')

In [ ]:
def perform_matching(df, treatment_var, year, note):
    """
    This function performs propensity score matching and returns a dataframe with both matched treatment and control observations.

    Parameters:
        df (pd.DataFrame): The dataframe for which propensity score matching is to be performed. 
                           This dataframe should already contain the propensity scores.
        treatment_var (str): The name of the treatment variable in df.

    Returns:
        df_matched (pd.DataFrame): A dataframe with matched treatment and control observations.
    """
    # Split the dataset into treatment and control groups
    treatment_data = df[df[treatment_var] == 1]
    control_data = df[df[treatment_var] == 0]
    
    # Create a NearestNeighbors matcher using the propensity scores
    matcher = NearestNeighbors(n_neighbors=1)
    matcher.fit(control_data['propensity_score'].values.reshape(-1, 1))
    
    # Find the nearest control observation for each treatment observation
    distances, indices = matcher.kneighbors(treatment_data['propensity_score'].values.reshape(-1, 1))

    # Create a matched dataset
    matched_control_indices = indices.flatten()
    matched_control_data = control_data.iloc[matched_control_indices]
    
    df_matched = pd.concat([treatment_data, matched_control_data]).reset_index(drop=True)
    
    # Add a column 'tt_status' to denote treatment and control status
    df_matched.loc[df_matched[treatment_var] == 1, 'tt_status'] = 'Treatment'
    df_matched.loc[df_matched[treatment_var] == 0, 'tt_status'] = 'Control'
    df_matched.to_parquet('../Data/processed/PSM/df_matched_{}_{}.parquet.gzip'.format(year,note), compression='gzip')
    
    return df_matched, treatment_data, control_data

In [ ]:
df_psm_matched_2017, treatment_2017, control_2017 =  perform_matching(df_psm_2017,'treatment','2017','unfiltered')
df_psm_matched_2018, treatment_2018, control_2018 =  perform_matching(df_psm_2018,'treatment','2018','unfiltered')
df_psm_matched_2019, treatment_2019, control_2019 =  perform_matching(df_psm_2019,'treatment','2019','unfiltered')
df_psm_matched_2020, treatment_2020, control_2020 =  perform_matching(df_psm_2020,'treatment','2020','unfiltered')
df_psm_matched_2021, treatment_2021, control_2021 =  perform_matching(df_psm_2021,'treatment','2021','unfiltered')

In [ ]:
df_psm_matched_2017_filtered, treatment_2017_filtered, control_2017_filtered =  perform_matching(df_psm_2017_filtered,'treatment','2017', 'filtered')
df_psm_matched_2018_filtered, treatment_2018_filtered, control_2018_filtered =  perform_matching(df_psm_2018_filtered,'treatment','2018', 'filtered')
df_psm_matched_2019_filtered, treatment_2019_filtered, control_2019_filtered =  perform_matching(df_psm_2019_filtered,'treatment','2019', 'filtered')
df_psm_matched_2020_filtered, treatment_2020_filtered, control_2020_filtered =  perform_matching(df_psm_2020_filtered,'treatment','2020', 'filtered')
df_psm_matched_2021_filtered, treatment_2021_filtered, control_2021_filtered =  perform_matching(df_psm_2021_filtered,'treatment','2021', 'filtered')

In [ ]:
def propensity_score_jitter_plot(treatment_data, control_data, matched_data):
    """
    This function generates a jitter plot of propensity scores for treatment and control groups before and after matching.

    Parameters:
        treatment_data (pd.DataFrame): The dataframe containing the original treatment group data.
        control_data (pd.DataFrame): The dataframe containing the original control group data.
        matched_data (pd.DataFrame): The dataframe containing the matched treatment and control group data.
    """
    
    # Split the dataset into different categories
    unmatched_treatment = treatment_data[~treatment_data.uuid.isin(matched_data.uuid)]
    unmatched_control = control_data[~control_data.uuid.isin(matched_data.uuid)]

    matched_treatment = treatment_data[treatment_data.uuid.isin(matched_data.uuid)]
    matched_control = control_data[control_data.uuid.isin(matched_data.uuid)]
    
    # Create a jitter plot for each category
    fig, ax = plt.subplots(figsize = (10,5))
    
    sns.stripplot(y=['Unmatched Treatment']*len(unmatched_treatment), x=unmatched_treatment['propensity_score'], jitter=0.2, alpha=0.01, color='black', marker='o', linewidth=0.1, ax=ax)
    sns.stripplot(y=['Matched Treatment']*len(matched_treatment), x=matched_treatment['propensity_score'], jitter=0.2, alpha=0.01, color='black', marker='o', linewidth=0.1, ax=ax)
    sns.stripplot(y=['Matched Control']*len(matched_control), x=matched_control['propensity_score'], jitter=0.2, alpha=0.01, color='black', marker='o', linewidth=0.1, ax=ax)
    sns.stripplot(y=['Unmatched Control']*len(unmatched_control), x=unmatched_control['propensity_score'], jitter=0.2, alpha=0.01, color='black', marker='o', linewidth=0.1, ax=ax)

    ax.set_ylabel('Propensity Score')
    ax.set_title('Jitter Plot of Propensity Scores')
    
    plt.show()

In [ ]:
propensity_score_jitter_plot(treatment_2017, control_2017, df_psm_matched_2017)
propensity_score_jitter_plot(treatment_2018, control_2018, df_psm_matched_2018)
propensity_score_jitter_plot(treatment_2019, control_2019, df_psm_matched_2019)
propensity_score_jitter_plot(treatment_2020, control_2020, df_psm_matched_2020)
propensity_score_jitter_plot(treatment_2021, control_2021, df_psm_matched_2021)

In [ ]:
propensity_score_jitter_plot(treatment_2017_filtered, control_2017_filtered, df_psm_matched_2017_filtered)
propensity_score_jitter_plot(treatment_2018_filtered, control_2018_filtered, df_psm_matched_2018_filtered)
propensity_score_jitter_plot(treatment_2019_filtered, control_2019_filtered, df_psm_matched_2019_filtered)
propensity_score_jitter_plot(treatment_2020_filtered, control_2020_filtered, df_psm_matched_2020_filtered)
propensity_score_jitter_plot(treatment_2021_filtered, control_2021_filtered, df_psm_matched_2021_filtered)

In [ ]:
df_psm_matched_2017[df_psm_matched_2017.treatment == 0].groupby('uuid', observed = True).size().sort_values()

### Diagnostics

In [ ]:
def propensity_score_histogram(treatment_data, control_data):
    """
    This function generates a histogram of propensity scores for treatment and control groups.

    Parameters:
        treatment_data (pd.DataFrame): The dataframe containing the treatment group data.
        control_data (pd.DataFrame): The dataframe containing the control group data.
    """
    
    plt.figure(figsize=(10, 5))
    plt.hist(control_data.propensity_score, bins=20, alpha=0.5, label='Control')
    plt.hist(treatment_data.propensity_score, bins=20, alpha=0.5, label='Treatment')
    plt.xlabel('Propensity Score')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()


In [ ]:
# propensity_score_histogram(treatment_2017, control_2017)
propensity_score_histogram(df_psm_matched_2017[df_psm_matched_2017['treatment'] == 1], df_psm_matched_2017[df_psm_matched_2017['treatment'] == 0])
propensity_score_histogram(df_psm_matched_2018[df_psm_matched_2018['treatment'] == 1], df_psm_matched_2018[df_psm_matched_2018['treatment'] == 0])
propensity_score_histogram(df_psm_matched_2019[df_psm_matched_2019['treatment'] == 1], df_psm_matched_2019[df_psm_matched_2019['treatment'] == 0])
propensity_score_histogram(df_psm_matched_2020[df_psm_matched_2020['treatment'] == 1], df_psm_matched_2020[df_psm_matched_2020['treatment'] == 0])
propensity_score_histogram(df_psm_matched_2021[df_psm_matched_2021['treatment'] == 1], df_psm_matched_2021[df_psm_matched_2021['treatment'] == 0])

In [ ]:
covariates = ['NBAGE', 'SEX_F', 'cds','ssep2','mean_ndvi', 'CANTON_ACRONYM_GE','CANTON_ACRONYM_VS']
figure_title = "Covariate Distributions Before and After Matching"
plot_covariate_distributions(df_psm_2021, df_psm_matched_2021, covariates, figure_title)

In [ ]:
balance_comparison = compare_balance(df_psm_2021, df_psm_matched_2021, categorical_columns+continuous_columns, treatment_var)
balance_comparison.plot()

In [ ]:
# plot_categorical_proportional_diff(data_scaled, df_matched, 'SEX_F', treatment_var)
# plot_categorical_proportional_diff(data_scaled, df_matched, 'CANTON_NAME_Genève', treatment_var)
# plot_categorical_proportional_diff(data_scaled, df_matched, 'CANTON_NAME_Valais', treatment_var)

# plot_categorical_proportional_diff(data_scaled, df_matched, 'ssep2', treatment_var)
# plot_categorical_proportional_diff(data_scaled, df_matched, 'cds', treatment_var)

In [ ]:
# Assuming you have the following data
variable_names = pd.DataFrame({"old": ["NBAGE", "ssep2", "SEX_F", "cds",'MTFRANCHISECOUV','LANG_FR','LANG_DE','CANTON_ACRONYM_GE','CANTON_ACRONYM_VS','E','N','E:N','D_MEDIC_B','D_MEDIC_S','mean_pm10','mean_no2','mean_pm25','mean_ndvi','mean_lst','mean_carnight'],
                               "new": ["Age", "SES index", "Sex", "CDS",'Franchise','French speaking','German speaking','Canton - Genève','Canton - Valais','E','N','E:N','Access to prim. care med.','Access to spec. med.','PM10','NO2','PM25','NDVI','LST','Nighttime car noise']})


# Calculate mean differences and proportions
mean_diffs_and_props = compute_mean_differences_and_proportions(df_psm_2017, df_psm_matched_2017, variable_names, treatment_var)
love_plot(mean_diffs_and_props, 0.1, 1)
mean_diffs_and_props = compute_mean_differences_and_proportions(df_psm_2021, df_psm_matched_2021, variable_names, treatment_var)
love_plot(mean_diffs_and_props, 0.1, 1)

Step 7: Assess the robustness of the results

To ensure the robustness of the estimated treatment effect, you can perform sensitivity analyses, such as varying the number of nearest neighbors or using alternative matching algorithms. This step helps confirm that your results are not overly sensitive to the specific matching approach used.

Here's an example of how to perform a sensitivity analysis by varying the number of nearest neighbors:

In [ ]:
from utils import sensitivity_analysis_k_neighbors

In [ ]:
# Run the sensitivity analysis for a range of k values
k_values = [1, 3, 5, 10, 20]
sensitivity_results = sensitivity_analysis_k_neighbors(df_psm_2017, treatment_2017, control_2017, continuous_columns+categorical_columns, treatment_var, outcome_var, k_values)
print(sensitivity_results)

In [ ]:
# Run the sensitivity analysis for a range of k values
k_values = [1, 3, 5, 10, 20]
sensitivity_results = sensitivity_analysis_k_neighbors(df_psm_2018, treatment_2018, control_2018, continuous_columns+categorical_columns, treatment_var, outcome_var, k_values)
print(sensitivity_results)

In [ ]:
# Run the sensitivity analysis for a range of k values
k_values = [1, 3, 5, 10, 20]
sensitivity_results = sensitivity_analysis_k_neighbors(df_psm_2019, treatment_2019, control_2019, continuous_columns+categorical_columns, treatment_var, outcome_var, k_values)
print(sensitivity_results)

In [ ]:
# Run the sensitivity analysis for a range of k values
k_values = [1, 3, 5, 10, 20]
sensitivity_results = sensitivity_analysis_k_neighbors(df_psm_2020, treatment_2020, control_2020, continuous_columns+categorical_columns, treatment_var, outcome_var, k_values)
print(sensitivity_results)

In [ ]:
# Run the sensitivity analysis for a range of k values
k_values = [1, 3, 5, 10, 20]
sensitivity_results = sensitivity_analysis_k_neighbors(df_psm_2021, treatment_2021, control_2021, continuous_columns+categorical_columns, treatment_var, outcome_var, k_values)
print(sensitivity_results)

This code performs the sensitivity analysis by estimating the ATE for different numbers of nearest neighbors (k). You can modify the k_values list to include other values or implement alternative matching algorithms to assess the robustness of your results further.

The Average Treatment Effect (ATE) represents the average difference in outcomes between the treatment and control groups in your matched dataset. An ATE of 1241.2 suggests that, on average, receiving the treatment is associated with an increase of 1241.2 units in the outcome variable compared to not receiving the treatment.

However, to provide a more meaningful interpretation, it's important to consider the context and the specific variables in your study. For example, if your study is evaluating the effect of a job training program (treatment) on annual income (outcome), an ATE of 1241.2 would imply that, on average, individuals who participated in the job training program earned $1,241.2 more per year than their matched counterparts who did not participate in the program.

Keep in mind that although propensity score matching helps control for observed covariates, it does not address unobserved confounders. Consequently, the ATE estimate should be interpreted as the average treatment effect on the treated (ATT) under the assumption of no unobserved confounding.

It's also essential to consider the quality of the matching, the robustness of the results, and any limitations in your study when interpreting the ATE.

There are several methods to estimate treatment effects, apart from the Average Treatment Effect (ATE) that we discussed earlier. Here are a few other popular treatment effect measures:

**Average Treatment Effect on the Treated (ATT)**: This measure calculates the average treatment effect for the individuals who received the treatment. It compares the outcomes of the treated group with their counterfactual outcomes if they had not received the treatment. It is especially useful when the focus is on understanding the impact of the treatment on those who were actually treated.

**Average Treatment Effect on the Controls (ATC)**: This measure calculates the average treatment effect for the individuals who did not receive the treatment. It compares the outcomes of the control group with their counterfactual outcomes if they had received the treatment. It helps to estimate the potential impact if the treatment were expanded to include those who were not treated initially.

**Local Average Treatment Effect (LATE)**: This measure estimates the treatment effect for a specific subpopulation, usually defined by an instrument variable. LATE is useful when the treatment effect is heterogeneous and the interest is in understanding the impact of the treatment on a particular subset of the population.

**Conditional Average Treatment Effect (CATE)**: This measure estimates the treatment effect for different subpopulations based on specific covariates or characteristics. It helps to understand the heterogeneous treatment effects across different groups, which can be useful for targeting interventions or identifying subpopulations that benefit the most or the least from the treatment.

Various estimation techniques can be applied to compute these treatment effect measures, such as matching, weighting, regression adjustment, and instrumental variable methods. The choice of method and treatment effect measure depends on the research question, data availability, and the assumptions that can be made about the data and the underlying causal relationships.

## Outcome analyses

### Do individuals using integrative medicine in year 1 have lower spending in the same year?

In [ ]:
ATE = df_psm_matched_2017.groupby(treatment_var)[outcome_var].mean().diff().iloc[-1]
print(f"Average Treatment Effect (ATE): {ATE}")

treated_outcomes = df_psm_matched_2017.loc[df_psm_matched_2017[treatment_var] == 1, outcome_var]
control_outcomes = df_psm_matched_2017.loc[df_psm_matched_2017[treatment_var] == 0, outcome_var]
ATT = treated_outcomes.mean() - control_outcomes.mean()
print(f"Average Treatment Effect on the Treated (ATT): {ATT}")

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = outcome_var, ax = ax)
# annotator = Annotator(chart, pairs=[('Treatment','Control')], data=df_matched, x='tt_status', y='NBAGE', order = list([i for i in df_matched['tt_status'].sort_values().unique()]))
# annotator.configure(test='Mann-Whitney', text_format='star', loc='inside', comparisons_correction="Bonferroni")
# annotator.apply_and_annotate()

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'PRESTATIONS_NETTES_AOS', ax = ax)
# annotator = Annotator(chart, pairs=[('Treatment','Control')], data=df_matched, x='tt_status', y='NBAGE', order = list([i for i in df_matched['tt_status'].sort_values().unique()]))
# annotator.configure(test='Mann-Whitney', text_format='star', loc='inside', comparisons_correction="Bonferroni")
# annotator.apply_and_annotate()

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'PRESTATIONS_BRUTES_LCA', ax = ax)
# annotator = Annotator(chart, pairs=[('Treatment','Control')], data=df_matched, x='tt_status', y='NBAGE', order = list([i for i in df_matched['tt_status'].sort_values().unique()]))
# annotator.configure(test='Mann-Whitney', text_format='star', loc='inside', comparisons_correction="Bonferroni")
# annotator.apply_and_annotate()

In [ ]:
# Calculate the average treatment effect (ATE)
ate = df_psm_matched_2017.groupby(treatment_var)[outcome_var].mean().diff().iloc[-1]
print("Average Treatment Effect:", ate)

### Do individuals using integrative medicine have better health outcomes in the same year?

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'n_inpatient_hosp', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'n_outpatient_hosp', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'time_to_rehosp_in', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'time_to_rehosp_out', ax=ax)

In [ ]:
df_psm_matched_2018['n_month_outpatienthosp'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'n_month_outpatienthosp', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'n_month_inpatienthosp', ax=ax)

In [ ]:
def get_ate_by_year(covariate):
    _dict = {}
    # Calculate the average treatment effect (ATE)
    for df, year in zip([df_psm_matched_2017, df_psm_matched_2018, df_psm_matched_2019, df_psm_matched_2020, df_psm_matched_2021], [2017, 2018, 2019, 2020, 2021]):
        ate = df.groupby(treatment_var)[covariate].mean().diff().iloc[-1]
        print(f"Average Treatment Effect {year}:", ate)
        _dict[year] = ate
    # Create a DataFrame
    df = pd.DataFrame(list(_dict.items()), columns=['Year', 'ATE'])
    return df

In [ ]:
dict_ate_inpatient_hosp = {}
# Calculate the average treatment effect (ATE)
for df, year in zip([df_psm_matched_2017, df_psm_matched_2018, df_psm_matched_2019, df_psm_matched_2020, df_psm_matched_2021], [2017, 2018, 2019, 2020, 2021]):
    ate = df.groupby(treatment_var)['n_month_inpatienthosp'].mean().diff().iloc[-1]
    print(f"Average Treatment Effect {year}:", ate)
    dict_ate_inpatient_hosp[year] = ate
# Create a DataFrame
df_ate_inhosp = pd.DataFrame(list(dict_ate_inpatient_hosp.items()), columns=['Year', 'Value'])

### Do individuals using integrative medicine have lower drug consumption in the same year?

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))

chart = sns.barplot(data = df_psm_matched_2021, x = 'tt_status', y = 'DRUGAMOUNT_BRUT', ax=ax)
# annotator = Annotator(chart, pairs=[('Treatment','Control')], data=df_matched, x='tt_status', y='DRUGAMOUNT_BRUT', order = list([i for i in df_matched['tt_status'].sort_values().unique()]))
# annotator.configure(test='Mann-Whitney', text_format='star', loc='inside', comparisons_correction="Bonferroni")
# annotator.apply_and_annotate()

In [ ]:
df_ate_drugamount = get_ate_by_year('DRUGAMOUNT_BRUT')

In [ ]:
df_ate_drugamount = get_ate_by_year('n_atc')

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'n_atc', ax=ax)

In [ ]:
# Calculate the average treatment effect (ATE)
ate = df_psm_matched_2017.groupby(treatment_var)['DRUGAMOUNT_BRUT'].mean().diff().iloc[-1]
print("Average Treatment Effect:", ate)

In [ ]:
# Calculate the average treatment effect (ATE)
ate = df_psm_matched_2017.groupby(treatment_var)['n_atc'].mean().diff().iloc[-1]
print("Average Treatment Effect:", ate)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'time_to_rehosp_in', ax=ax)

In [ ]:
df_psm_matched_2017.loc[df_psm_matched_2017.time_to_rehosp_out == 0, 'time_to_rehosp_out'] = np.nan
df_psm_matched_2017.loc[df_psm_matched_2017.time_to_rehosp_in == 0, 'time_to_rehosp_out'] = np.nan

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'time_to_rehosp_out', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
chart = sns.barplot(data = df_psm_matched_2017, x = 'tt_status', y = 'time_to_rehosp_in', ax=ax)

In [ ]:
# Calculate the average treatment effect (ATE)
ate = df_psm_matched_2017.groupby(treatment_var)['time_to_rehosp_out'].mean().diff().iloc[-1]
print("Average Treatment Effect:", ate)

In [ ]:
df_psm_matched_2017.time_to_rehosp_out.value_counts()

### Do individuals using integrative medicine in year 1 have lower spending in following years?

### Do individuals using integrative medicine in year 1 have lower drug consumption in following years?

### Do individuals using integrative medicine in year 1 have better health outcomes in the following years?

### Do individuals using integrative medicine in year 1 have better CDS in the following years?

### Is there an effect of the number of years in the treatment group ?

## PSM on a continuous treatment variable
Implementation using Python using GPT 4 below, documentation from a CRAN package on this link https://cran.r-project.org/web/packages/CBPS/index.html

In [ ]:
#Propensity Score Matching (PSM) with a continuous treatment variable requires a different approach compared to binary treatment. One common method is Generalized Propensity Score (GPS) matching. The GPS is the conditional density of receiving a particular treatment level given the observed covariates. Here's a step-by-step guide using Python:

#Import Libraries: Import necessary Python libraries.
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

#Prepare Data: Assume df is your DataFrame and treatment is the continuous treatment variable, and covariates are your control variables.
covariates = df[['covariate_1', 'covariate_2', 'covariate_3']]
treatment = df['treatment']
#Estimate Propensity Scores: Fit a regression model to estimate propensity scores. The Linear Regression model works well for continuous treatments.
model = LinearRegression()
model.fit(covariates, treatment)
propensity_scores = model.predict(covariates)
#Calculate GPS: You can calculate the generalized propensity score. For simplicity, we can consider it as the density of the predicted propensity score, which can be calculated using the Normal distribution in this example.

gps_scores = norm.pdf(propensity_scores, np.mean(propensity_scores), np.std(propensity_scores))
#Match: Now, you can match the units based on the calculated GPS. Several matching techniques like nearest neighbor, caliper matching, etc., can be employed. You may need to implement this step manually or use specialized libraries.

## Panel data modelling

In [ ]:
# Required Libraries
import pandas as pd
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from statsmodels.datasets import grunfeld

In [ ]:
# Required Libraries
data = grunfeld.load_pandas().data
# Loading Grunfeld Investment data
year = pd.Categorical(data.year)

# Preparing data for panel model
data = data.set_index(['firm','year'])
data['year'] = year

# Pooled OLS model
pols = PanelOLS.from_formula('invest ~ value + capital + EntityEffects', data=data)
pols_result = pols.fit()
print(pols_result)

# Fixed effects model
fem = PanelOLS.from_formula('invest ~ value + capital + EntityEffects', data=data)
fem_result = fem.fit()
print(fem_result)

# Random effects model
rem = RandomEffects.from_formula('invest ~ value + capital', data=data)
rem_result = rem.fit()
print(rem_result)


# Legacy code - Other PSM approaches

## Matching without replacement

In [ ]:
# Split the dataset into treatment and control groups
treatment_data_wo_replacement = data_scaled[data_scaled[treatment_var] == 1]
control_data_wo_replacement = data_scaled[data_scaled[treatment_var] == 0]
# Set an initial large distance for each control individual
control_data_wo_replacement['distance'] = np.inf

# Set the index to be the original row order for later identification
control_data_wo_replacement['index'] = range(len(control_data_wo_replacement))

for i in range(len(treatment_data_wo_replacement)):
    # Calculate the absolute distance between the treatment individual and all control individuals
    distances_wo_replacement = abs(treatment_data_wo_replacement.iloc[i]['propensity_score'] - control_data_wo_replacement['propensity_score'])
    
    # If the minimum distance is less than the current saved distance for that control individual, update the match
    if distances_wo_replacement.min() < control_data_wo_replacement.loc[distances_wo_replacement.idxmin(), 'distance']:
        control_data_wo_replacement.loc[distances_wo_replacement.idxmin(), 'distance'] = distances_wo_replacement.min()
        control_data_wo_replacement.loc[distances_wo_replacement.idxmin(), 'match'] = i

# Select only those control individuals that have a match
matched_control_data_wo_replacement = control_data_wo_replacement.dropna(subset=['match'])

# Create the matched dataset
df_matched_wo_replacement = pd.concat([treatment_data_wo_replacement, matched_control_data_wo_replacement]).reset_index(drop=True)

In [ ]:
df_matched_wo_replacement.loc[df_matched_wo_replacement['treatment'] == 1, 'tt_status'] = 'Treatment'
df_matched_wo_replacement.loc[df_matched_wo_replacement['treatment'] == 0, 'tt_status'] = 'Control'

### Adding a caliper

From the results of it, looks quite useless

In [ ]:
# Compute absolute differences between treatment and control propensities
abs_diff_propensity = np.abs(treatment_data['propensity_score'].values.reshape(-1, 1) - control_data.iloc[matched_control_indices]['propensity_score'].values.reshape(-1, 1))

# Define caliper
caliper = 0.05

# Apply caliper: we only keep the pairs for which the absolute difference in propensity score is below the caliper
indices_within_caliper = abs_diff_propensity.flatten() < caliper

# Filter treatment and control data
treatment_data_caliper = treatment_data[indices_within_caliper]
matched_control_data_caliper = matched_control_data[indices_within_caliper]

# Concatenate treatment and control data to get the final matched dataset
df_matched_caliper = pd.concat([treatment_data_caliper, matched_control_data_caliper]).reset_index(drop=True)
df_matched_caliper = pd.merge(df_matched_caliper, data_2017[['uuid', outcome_var]], on='uuid', how='left')

## Matching with replacement

Use the `NearestNeighbors` algorithm from `scikit-learn` to perform 1-to-1 nearest-neighbor matching based on the propensity scores. Then, create a matched dataset containing the matched treatment and control observations.

In [ ]:
# Split the dataset into treatment and control groups
treatment_data = data_scaled[data_scaled[treatment_var] == 1]
control_data = data_scaled[data_scaled[treatment_var] == 0]
# Create a NearestNeighbors matcher using the propensity scores
matcher = NearestNeighbors(n_neighbors=1)
matcher.fit(control_data['propensity_score'].values.reshape(-1, 1))
# Find the nearest control observation for each treatment observation
distances, indices = matcher.kneighbors(treatment_data['propensity_score'].values.reshape(-1, 1))

# Create a matched dataset
matched_control_indices = indices.flatten()
matched_control_data = control_data.iloc[matched_control_indices]
df_matched = pd.concat([treatment_data, matched_control_data]).reset_index(drop=True)
# df_matched = pd.merge(df_matched, data_2017[['uuid',outcome_var]], on = 'uuid', how='left')
df_matched.loc[df_matched['treatment'] == 1, 'tt_status'] = 'Treatment'
df_matched.loc[df_matched['treatment'] == 0, 'tt_status'] = 'Control'

In [ ]:
### STORING MATCHES

# Split the dataset into treatment and control groups
treatment_data = data_scaled[data_scaled[treatment_var] == 1]
control_data = data_scaled[data_scaled[treatment_var] == 0]

# Create a NearestNeighbors matcher using the propensity scores
matcher = NearestNeighbors(n_neighbors=1)
matcher.fit(control_data['propensity_score'].values.reshape(-1, 1))
# Find the nearest control observation for each treatment observation
distances, indices = matcher.kneighbors(treatment_data['propensity_score'].values.reshape(-1, 1))

# Create a matched dataset
# matched_control_indices = indices.flatten()
matched_control_uuids = control_data.iloc[indices.flatten()]['uuid'].values
# matched_control_data = control_data.iloc[matched_control_indices]
# treatment_data = treatment_data.assign(matched_pair=np.arange(treatment_data.shape[0]))
treatment_data['matched_uuid'] = matched_control_uuids

matched_control_data = matched_control_data.assign(matched_pair=np.arange(matched_control_data.shape[0]))

# df_matched = pd.concat([treatment_data, matched_control_data]).reset_index(drop=True)
df_matched_w_match = treatment_data.merge(control_data, left_on='matched_uuid', right_on='uuid', suffixes=('_treatment', '_control'))


## Sensitivity analysis using causalml

An alternative approach to propensity score matching is Coarsened Exact Matching (CEM). CEM is an orthogonal matching method that temporarily coarsens the data by placing observations into strata based on their covariate values and then performs exact matching on the coarsened data.

In [ ]:
import causalml
from causalml.match import NearestNeighborMatch, MatchOptimizer
from causalml.inference.meta import LRSRegressor
from sklearn.linear_model import LinearRegression
def coarsened_exact_matching(treatment_data, control_data, treatment_var, covariate_columns, bins=5):
    n = len(treatment_data) + len(control_data)
    data = pd.concat([treatment_data, control_data])

    # Coarsen the covariates
    coarsened_covariates = pd.DataFrame(index=data.index, columns=covariate_columns)
    
    for col in covariate_columns:
        coarsened_covariates[col] = pd.cut(data[col], bins=bins, labels=False)
    
    # Create a matcher and find the matched pairs
    matcher = NearestNeighborMatch(replace=False, ratio=1, random_state=42)
    matches = matcher.match(data.loc[:, treatment_var], [treatment_var], coarsened_covariates)
    
    # Extract the matched dataset
    matched_data = data.loc[matches.index]
    
    return matched_data

# Perform CEM on the treatment and control data
# cem_matched_data = coarsened_exact_matching(treatment_data, control_data, treatment_var, covariate_columns, bins=5)

Check the quality of the matching by comparing the balance of covariates between the treatment and control groups in the CEM-matched dataset:

In [ ]:
balance_comparison_cem = compare_balance(data_scaled, cem_matched_data, covariate_columns, treatment_var)
print(balance_comparison_cem)

## Using pymatch

In [ ]:
test = df_sample[df_sample.treatment == 1]
control = df_sample[df_sample.treatment == 0]

In [ ]:
m = Matcher(test, control, yvar="treatment", exclude=[])

In [ ]:
# for reproducibility
np.random.seed(42)

m.fit_scores(balance=True, nmodels=10)

In [ ]:
m.predict_scores()

## Using `psmpy`

This script assumes that your data is in a CSV file named "data.csv" and that the variable indicating treatment status is called "treatment" (with 1 indicating the treatment group and 0 indicating the control group). The script also assumes that the variables you want to use for propensity score estimation are "age", "gender", and "income".

The script first uses the PropensityScoreEstimator class to estimate the propensity scores for each individual in the treatment and control groups based on the specified covariates. It then uses the PropensityScoreMatching class to match individuals from the treatment and control groups based on their estimated propensity scores. Finally, it uses the balance_table function to examine the balance of covariates in the matched sample.

You can also use other matching methods like Nearest Neighbor, Radius, and Kernel methods.

In [ ]:
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *
import sys
# import pymatch
# from pymatch.Matcher import Matcher

In [ ]:
psm = PsmPy(data_scaled, treatment='treatment', indx='uuid', exclude = [], seed = 42)

psm.logistic_ps(balance = True)

psm.predicted_data

psm.knn_matched(matcher='propensity_logit', replacement=True, caliper=0.05, drop_unmatched=False)

# psm.knn_matched_12n(matcher='propensity_logit', how_many=2)

psm.plot_match(Title='Side by side matched controls', Ylabel='Number of individuals', Xlabel= 'Propensity logit', names = ['treatment', 'control'], save=True)

In [ ]:
df_matched_psmpy=psm.df_matched

In [ ]:
psm.effect_size_plot(save=False)

In [ ]:
C_COLOUR = 'grey'
T_COLOUR = 'green'
C_LABEL = 'Control'
T_LABEL = 'Treatment'
for var in ['NBAGE', 'SEX_F','MTFRANCHISECOUV','CANTON_NAME_Genève']:
    fig, ax = plt.subplots(1,2,figsize=(10,4))
    # Visualise original distribution
    sns.kdeplot(data=df_sample[df_sample['treatment'] == 0], x=var, fill=True, 
                color=C_COLOUR, label=C_LABEL, ax=ax[0])
    sns.kdeplot(data=df_sample[df_sample['treatment'] == 1], x=var, fill=True, 
                color=T_COLOUR, label=T_LABEL, ax=ax[0])
    ax[0].set_title('Before matching')
    
    # Visualise new distribution
    sns.kdeplot(data=df_matched[df_matched['treatment'] == 0], x=var, 
                fill=True, color=C_COLOUR, label=C_LABEL, ax=ax[1])
    sns.kdeplot(data=df_matched[df_matched['treatment'] == 1], x=var, 
                fill=True, color=T_COLOUR, label=T_LABEL, ax=ax[1])
    ax[1].set_title('After matching')
    ax[1].set_ylabel("")
    plt.tight_layout()
ax[0].legend(loc='center', bbox_to_anchor=(1.1, -0.3));

In [ ]:
df_matched = pd.merge(df_matched, data_2017[['uuid',outcome_var]].sample(20000, random_state=42), on = 'uuid')

In [ ]:
# Calculate the average treatment effect (ATE)
ate = df_matched.groupby(treatment_var)[outcome_var].mean().diff().iloc[-1]
print("Average Treatment Effect:", ate)

## Method 2 - Package `DoWhy`

In [ ]:
# Import necessary libraries
import dowhy
from dowhy import CausalModel
import matplotlib.pyplot as plt


# Specify treatment variable and covariates
df_sample = data_2017[['uuid','treatment','NBAGE','MTFRANCHISECOUV','SEX_F','PRESTATIONS_TOTAL']].sample(2000, random_state=42)

# Define causal model using DoWhy
model = CausalModel(
    data=df_sample,
    treatment='treatment',
    outcome='PRESTATIONS_TOTAL',
    common_causes=['NBAGE', 'SEX_F']
)
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
#print(estimate)
print("The Causal Estimate is " + str(estimate.value))

In [ ]:
model.view_model()

In [ ]:
from IPython.display import Image, display
display(Image(filename="causal_model.png"))

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
causal_estimate_match = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.propensity_score_matching",
                                              target_units="atc", method_params={
        'ratio': 1.0,
        'matching_method': 'nearest',
        'distance_metric': 'absolute_difference',
    })
print(causal_estimate_match)
print("Causal Estimate is " + str(causal_estimate_match.value))

In [ ]:
# Plot propensity score distribution before and after matching
propensity_score_data = df_sample.propensity_score
plt.figure(figsize=(10, 5))
plt.hist(df_sample[df_sample['treatment'] == 0].propensity_score, bins=20, alpha=0.5, label='Control')
plt.hist(df_sample[df_sample['treatment'] == 1].propensity_score, bins=20, alpha=0.5, label='Treated')
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()


In [ ]:
# Import necessary libraries
import dowhy
from dowhy import CausalModel
import matplotlib.pyplot as plt
from dowhy.do_samplers import propensity_score_matching_estimator


# Specify treatment variable and covariates

# Define causal model using DoWhy
model = CausalModel(
    data=data,
    treatment=treatment,
    outcome='outcome',
    common_causes=covariates
)

# Estimate propensity scores using logistic regression
propensity_score_estimator = propensity_score_matching_estimator(
    data=data,
    treatment_variable=treatment,
    outcome_variable='outcome',
    method='lr'
)
propensity_score_estimator.reset()
propensity_scores = propensity_score_estimator.estimate()
data['propensity_score'] = propensity_scores['fitted']
# Perform matching
estimate = model.estimate_effect(
    method_name='backdoor.propensity_score_matching',
    target_units='ate',
    method_params={
        'ratio': 1.0,
        'matching_method': 'nearest',
        'distance_metric': 'absolute_difference',
    }
)

# Print average treatment effect estimate
print("ATE estimate: ", estimate.value)

# Plot propensity score distribution before and after matching
propensity_score_data = model.propensity_score
plt.figure(figsize=(10, 5))
plt.hist(propensity_score_data[data[treatment] == 0], bins=20, alpha=0.5, label='Control')
plt.hist(propensity_score_data[data[treatment] == 1], bins=20, alpha=0.5, label='Treated')
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()


## Method 3 - Package `causalinference`

In [ ]:
data_2017.columns[:50]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score
from causalinference import CausalModel

In [ ]:
# Construct a CausalModel
cm = CausalModel(
    Y=data_2017[outcome_var].values, 
    D=data_scaled[treatment_var].values, 
    X=data_scaled[categorical_columns+continuous_columns].values
)

# Estimate propensity scores
cm.est_propensity()

# Perform matching with caliper
cm.trim_s()
cm.stratify_s()
# cm.match_s()
cm.est_via_matching(matches=1, bias_adj=True)

# Print matched data summary
print(cm.summary_stats)

In [ ]:
y = data_2017['PRESTATIONS_TOTAL'].values
t = data_2017['treatment'].values
X = data_2017[['SEX_F', 'NBAGE','MTFRANCHISECOUV']]
X = pd.DataFrame(StandardScaler().fit_transform(X), 
                 columns=X.columns).values
model = CausalModel(y, t, X)
model.est_via_matching()
print(model.estimates)

In [ ]:
for var in ['logit', 'age']:
    fig, ax = plt.subplots(1,2,figsize=(10,4))
    # Visualise original distribution
    sns.kdeplot(data=df[~df[TREATMENT]], x=var, shade=True, 
                color=C_COLOUR, label=C_LABEL, ax=ax[0])
    sns.kdeplot(data=df[df[TREATMENT]], x=var, shade=True, 
                color=T_COLOUR, label=T_LABEL, ax=ax[0])
    ax[0].set_title('Before matching')
    
    # Visualise new distribution
    sns.kdeplot(data=matched_df[~matched_df[TREATMENT]], x=var, 
                shade=True, color=C_COLOUR, label=C_LABEL, ax=ax[1])
    sns.kdeplot(data=matched_df[matched_df[TREATMENT]], x=var, 
                shade=True, color=T_COLOUR, label=T_LABEL, ax=ax[1])
    ax[1].set_title('After matching')
    ax[1].set_ylabel("")
    plt.tight_layout()
ax[0].legend(loc='center', bbox_to_anchor=(1.1, -0.3));

## Optimising the procedure

Perform KNN matching.

psm.knn_matched(matcher='propensity_logit', replacement=False, caliper=None, drop_unmatched=True)

Note:

matcher - propensity_logit (default) and generated in previous step alternative option is propensity_score, specifies the argument on which matching will proceed

replacement - False (default), determines whethermacthing will happen with or without replacement,when replacement is false matching happens 1:1

caliper - None (default), user can specify caliper size relative to std. dev of the control sample, restricting neighbors eligible to match within a certain distance.

drop_unmatched - True (default) In the event that indexes do not have a match due to caliper size it will remove them from the 'matched_df', 'matched_ids' and subsequent calculations of effect size

In [ ]:
# Use a larger sample size for propensity score estimation
sample_size = 2000

# Increase the number of neighbors used for matching
k = 3

# Use a larger caliper
caliper = 0.2

# Use decision tree to estimate the propensity scores
psm = PsmPy(data_2017[['uuid','treatment','NBAGE','MTFRANCHISECOUV','SEX_F']].sample(sample_size),
            treatment='treatment', indx='uuid', exclude=[])

psm.logistic_ps(balance=True)

psm.knn_matched(replacement=False, caliper=caliper)

psm.plot_match(Title='Side by side matched controls', Ylabel='Number of individuals',
               Xlabel='Propensity score', names=['treatment', 'control'], save=True)

In [ ]:
psm.effect_size_plot(save=False)

## Procedure with XGBoost to predict scores

In [ ]:
import xgboost as xgb

# Define the treatment and outcome variables
treatment = 'treatment'
outcome = 'outcome'

# Define the covariates
covariates = ['NBAGE','MTFRANCHISECOUV','SEX_F']

# Split the data into treatment and control groups
treated = data_2017[data_2017[treatment] == 1]
control = data_2017[data_2017[treatment] == 0]

# Sample the data to improve the training speed
treated = treated.sample(frac=0.1, replace=False, random_state=1)
control = control.sample(frac=0.1, replace=False, random_state=1)

# Create the training and testing datasets
train = pd.concat([treated, control], axis=0)
test = data.drop(train.index)

# Create the XGBoost data matrices
dtrain = xgb.DMatrix(train[covariates], label=train[treatment])
dtest = xgb.DMatrix(test[covariates])

# Define the XGBoost hyperparameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'seed': 1
}

# Train the XGBoost model
bst = xgb.train(params, dtrain)

# Predict the propensity scores
train['propensity_score'] = bst.predict(dtrain)
test['propensity_score'] = bst.predict(dtest)

In [ ]:
matched_data

In [ ]:
# Define the caliper value for matching
caliper = 0.05

# Combine the treated and control groups
matched_data = pd.concat([treated, control], axis=0)

# Create a PsmPy object with the matched data
psm = PsmPy(matched_data[['uuid','treatment','NBAGE','MTFRANCHISECOUV','SEX_F']], treatment=treatment, indx='uuid', exclude=[])

# Perform nearest-neighbor matching using the predicted propensity scores
psm.knn_matched(replacement=False, caliper=caliper)

# Plot the matched data
psm.plot_match(Title='Side by side matched controls', Ylabel='Number of individuals',
               Xlabel='Propensity score', names=['treatment', 'control'], save=True)

# Compute the effect size
effect_size = cohenD(psm.data[treatment], psm.data[outcome], psm.data['Matched'])

# Print the effect size
print('Effect size:', effect_size)

## R MatchIt implementation

In [ ]:
import os
os.environ['R_HOME'] = '/Users/david/miniforge3/envs/py310/lib/R'

In [ ]:
# import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

In [ ]:
import pandas as pd
import numpy as np
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Load the MatchIt package in R
matchit = importr('MatchIt')

# Load the data into R
robjects.globalenv['data'] = robjects.conversion.py2rpy(data_2017)

# Define the treatment and outcome variables
treatment = 'treatment'
outcome = 'outcome'

# Define the covariates
covariates = ['NBAGE','MTFRANCHISECOUV','SEX_F']
robjects.globalenv['covariates'] = robjects.StrVector(covariates)

# Define the R script to perform propensity score matching
script = """
# Load the data into R
install.packages('Matchit')
data <- data.frame(data)

# Define the treatment and outcome variables
treatment <- '""" + treatment + """'
outcome <- '""" + outcome + """'

# Define the covariates
covariates <- covariates

# Perform propensity score matching using the nearest neighbor method with a caliper
matched_data <- matchit(
    formula = as.formula(paste(treatment, "~", paste(covariates, collapse="+"))),
    data = data,
    method = "nearest",
    caliper = 0.05
)

# Extract the matched data from the "matched" object
matched_data <- match.data(matched_data)

# Return the matched data as a data frame
matched_data <- data.frame(matched_data)
"""

# Run the R script to perform propensity score matching
robjects.r(script)

# Retrieve the matched data from R and convert it to a pandas dataframe
matched_data = pd.DataFrame(np.array(robjects.globalenv['matched_data']), columns=data.columns)


In [ ]:
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Load the MatchIt package
matchit = importr('MatchIt')

# Load the Lalonde dataset from the MatchIt package
lalonde = matchit.Lalonde

# Create a formula for the treatment variable and covariates
formula = robjects.Formula('treat ~ age + educ + black + hisp + married + nodegr + re74 + re75')

# Perform matching using the nearest neighbor method
matched_data = matchit.matchit(formula=formula, data=lalonde, method='nearest', ratio=1)

# View the matched data
print(matched_data)
